In [ ]:
import torch
import argparse
from random import randint
from omegaconf import OmegaConf
from pytorch_lightning import seed_everything
from torch import autocast
from diffusion.util import instantiate_from_config
from diffusion import ddpm
from contextlib import nullcontext


precision = "autocast"
device = "cuda"

## JIT compilation of UNet model 
n_iter = 1
batch_size = 1
config = "diffusion/v1-inference.yaml"
config = OmegaConf.load(f"{config}")
Height = 128
Width = 128
scale = 7.5 
seed = 1245
ddim_eta = 0.0
small_batch = "False"
turbo = True
from_file = False
n_samples = 1
unet_bs = 1
fixed_code = False
n_rows = 1
sampler = "plms"


if seed == None:
    seed = randint(0, 1000000)
seed_everything(seed)


model = instantiate_from_config(config.modelUNet)

if device == "cuda" and precision == "autocast":
    modelsd  = torch.load("split_model/model1.4_fp16.pth")
else:
    modelsd  = torch.load("split_model/model1.4_fp32.pth")


# model.model1 = ddpm.DiffusionWrapper(model.unetConfigEncode)
# model.model2 = ddpm.DiffusionWrapperOut(model.unetConfigDecode)
_, _ = model.load_state_dict(modelsd['state_dict'] , strict=False)


model

In [ ]:
state = {'state_dict': model.state_dict()}
torch.save(state, "split_model/modelFS1.4_fp32_betas.pth")

In [1]:
# "disable-hardware-acceleration": true

In [2]:
image_path = "testImages/dog.png"
mask_path = "testImages/dog_mask.png"
n_iter = 1
batch_size = 1
Height = 128
Width = 128
scale = 7.5 
seed = 1245
ddim_eta = 0.0
small_batch = "False"
turbo = True
from_file = False
n_samples = 1
precision = "autocast"
outdir = "outputs/inpaint-samples"
device = "cuda"
prompt = "A bear surfing in the ocean"
unet_bs = 1
fixed_code = False
n_rows = 1
sampler = "plms"

In [3]:
import argparse, os, re
import torch
import numpy as np
from random import randint
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange
from torchvision.utils import make_grid
import time
from pytorch_lightning import seed_everything
from torch import autocast
from contextlib import contextmanager, nullcontext
from diffusion.util import instantiate_from_config
from diffusion.util import split_weighted_subprompts, logger
from transformers import logging
# from samplers import CompVisDenoiser
logging.set_verbosity_error()


def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


def load_model_from_config(ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    return sd


config = "diffusion/v1-inference.yaml"
DEFAULT_CKPT = "models/diffusion/stable-diffusion-v1/model.ckpt"

tic = time.time()
os.makedirs(outdir, exist_ok=True)
outpath = outdir
grid_count = len(os.listdir(outpath)) - 1

if seed == None:
    seed = randint(0, 1000000)
seed_everything(seed)

config = OmegaConf.load(f"{config}")

Global seed set to 1245


In [4]:
model = instantiate_from_config(config.modelUNet)
modelsd  = torch.load("split_model/model1.4_fp16_betas.pth")
_, _ = model.load_state_dict(modelsd['state_dict'] , strict=False)
model.model1  = torch.jit.load("model1_traced.pth")
model.model2 = torch.jit.load("model2_traced.pth")
model.model2.eval()
model.model1.eval()
_ = model.half()

UNet: Running in eps-prediction mode


In [5]:
torch.cuda.memory_allocated() / 1e6

0.0

In [7]:
import torch
import torch_tensorrt
torch_tensorrt.compile()

ModuleNotFoundError: No module named 'torch_tensorrt'

In [21]:
_ = model.model1.to(device)

In [25]:
_ = model.model2.to(device)

In [10]:
_ = model.to(device)

In [23]:
_ = model.to("cpu")

In [9]:
torch.cuda.memory_allocated() / 1e6

0.0

In [4]:
# model = instantiate_from_config(config.modelUNet)
# modelsd  = torch.load("split_model/model1.4_fp16.pth")
# _, _ = model.load_state_dict(modelsd['state_dict'], strict=False)
# model.half()

In [6]:
model = instantiate_from_config(config.modelUNet)
modelsd  = torch.load("split_model/model1.4_fp16_betas.pth")
_, _ = model.load_state_dict(modelsd['state_dict'], strict=True)

UNet: Running in eps-prediction mode


In [6]:
# state = {'state_dict': model.state_dict()}
# torch.save(state, "split_model/model1.4_fp16_betas.pth")

In [ ]:
# JIT model 1
model.to(device)
inp = torch.rand([1, 4, 64, 64]).to(device), torch.tensor([981]).to(device), torch.rand([1, 77, 768]).to(device)

precision_scope = autocast
with precision_scope("cuda"):

    m = torch.jit.trace(model.model1, inp)
    
torch.jit.save(m, "model1_traced.pth")


# JIT model2

ten =  [torch.rand([1, 320, 32, 32]).to(device),
        torch.rand([1, 320, 32, 32]).to(device),
        torch.rand([1, 320, 32, 32]).to(device),
        torch.rand([1, 320, 16, 16]).to(device),
        torch.rand([1, 640, 16, 16]).to(device),
        torch.rand([1, 640, 16, 16]).to(device),
        torch.rand([1, 640, 8, 8]).to(device),
        torch.rand([1, 1280, 8, 8]).to(device),
        torch.rand([1, 1280, 8, 8]).to(device),
        torch.rand([1, 1280, 4, 4]).to(device),
        torch.rand([1, 1280, 4, 4]).to(device),
        torch.rand([1, 1280, 4, 4]).to(device)]
inp = (torch.rand([1, 1280, 4, 4]).to(device), torch.rand([1, 1280]).to(device),ten, torch.rand([1, 77, 768]).to(device))

precision_scope = autocast
with precision_scope("cuda"):

    m = torch.jit.trace(model.model2, inp)

torch.jit.save(m, "model2_traced.pth")


In [ ]:
modelFS = instantiate_from_config(config.modelFirstStage)
modelFSsd  = torch.load("split_model/modelFS1.4_fp16.pth")
_, _ = modelFS.load_state_dict(modelFSsd['state_dict'], strict=False)
modelFS.eval()


model = instantiate_from_config(config.modelUNet)
modelsd  = torch.load("split_model/model1.4_fp16.pth")
_, _ = model.load_state_dict(modelsd['state_dict'] , strict=False)
model.eval()
model.unet_bs = unet_bs
model.cdevice = device
model.turbo = turbo

modelCS = instantiate_from_config(config.modelCondStage)
modelCSsd  = torch.load("split_model/modelCS1.4_fp16.pth")
_, _ = modelCS.load_state_dict(modelCSsd['state_dict'], strict=False)
modelCS.eval()
modelCS.cond_stage_model.device = device

del modelCSsd
del modelFSsd
del modelsd


if device != "cpu" and precision == "autocast":
    model.half()
    # modelFS.half()
    modelCS.half()

In [ ]:
start_code = None
if fixed_code:
    start_code = torch.randn([n_samples, C, H // f, W // f], device=device)


batch_size = n_samples
n_rows = n_rows if n_rows > 0 else batch_size
if not from_file:
    assert prompt is not None
    prompt = prompt
    print(f"Using prompt: {prompt}")
    data = [batch_size * [prompt]]

else:
    print(f"reading prompts from {from_file}")
    with open(from_file, "r") as f:
        text = f.read()
        print(f"Using prompt: {text.strip()}")
        data = text.splitlines()
        data = batch_size * list(data)
        data = list(chunk(sorted(data), batch_size))


if precision == "autocast" and device != "cpu":
    precision_scope = autocast
else:
    precision_scope = nullcontext

seeds = ""
with torch.no_grad():

    all_samples = list()
    for n in trange(n_iter, desc="Sampling"):
        for prompts in tqdm(data, desc="data"):

            sample_path = os.path.join(outpath, "_".join(re.split(":| ", prompts[0])))[:150]
            os.makedirs(sample_path, exist_ok=True)
            base_count = len(os.listdir(sample_path))

            with precision_scope("cuda"):
                modelCS.to(device)
                uc = None
                if scale != 1.0:
                    uc = modelCS.get_learned_conditioning(batch_size * [""])
                if isinstance(prompts, tuple):
                    prompts = list(prompts)

                subprompts, weights = split_weighted_subprompts(prompts[0])
                if len(subprompts) > 1:
                    c = torch.zeros_like(uc)
                    totalWeight = sum(weights)
                    # normalize each "sub prompt" and add it
                    for i in range(len(subprompts)):
                        weight = weights[i]
                        # if not skip_normalize:
                        weight = weight / totalWeight
                        c = torch.add(c, modelCS.get_learned_conditioning(subprompts[i]), alpha=weight)
                else:
                    c = modelCS.get_learned_conditioning(prompts)

                shape = [n_samples, C, H // f, W // f]

                if device != "cpu":
                    mem = torch.cuda.memory_allocated() / 1e6
                    modelCS.to("cpu")
                    while torch.cuda.memory_allocated() / 1e6 >= mem:
                        time.sleep(1)

                samples_ddim = model.sample(
                    S=ddim_steps,
                    conditioning=c,
                    seed=seed,
                    shape=shape,
                    verbose=False,
                    unconditional_guidance_scale=scale,
                    unconditional_conditioning=uc,
                    eta=ddim_eta,
                    x_T=start_code,
                    sampler = sampler,
                )

                modelFS.to(device)

                print(samples_ddim.shape)
                print("saving images")
                for i in range(batch_size):

                    x_samples_ddim = modelFS.decode_first_stage(samples_ddim[i].unsqueeze(0))
                    x_sample = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
                    x_sample = 255.0 * rearrange(x_sample[0].cpu().numpy(), "c h w -> h w c")
                    Image.fromarray(x_sample.astype(np.uint8)).save(
                        os.path.join(sample_path, "seed_" + str(seed) + "_" + f"{base_count:05}.{format}")
                    )
                    seeds += str(seed) + ","
                    seed += 1
                    base_count += 1

                if device != "cpu":
                    mem = torch.cuda.memory_allocated() / 1e6
                    modelFS.to("cpu")
                    while torch.cuda.memory_allocated() / 1e6 >= mem:
                        time.sleep(1)
                del samples_ddim
                print("memory_final = ", torch.cuda.memory_allocated() / 1e6)

toc = time.time()
time_taken = (toc - tic) / 60.0

print(
    (
        "Samples finished in {0:.2f} minutes and exported to "
        + sample_path
        + "\n Seeds used = "
        + seeds[:-1]
    ).format(time_taken)
)
